In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image
from sklearn import linear_model

In [7]:
!python -m pip install pooch # You might need to run this once  
                             # and then you can delete it..  
                             # Needed for Problem 3.  

## Background

So things probably went a little quick via the Zoom lecture, so we'll collect some critical information here that we'll use later in the assignment.  

> **Dictionary Learning**: Given data $n\times m$ real-value data matrix $Y$, we want to find a $n\times p$ _dictionary_ $A_{\ast}$ and sparse encoding of $Y$, say $p\times m$ $Z_{\ast}$ which solves
$$
A_{\ast}, Z_{\ast} = \text{arg min}_{A, Z} \left|\left|Y-AZ\right|\right|_{F} + \lambda \left|\left|Z\right|\right|_{1}, ~ \lambda > 0.
$$
Each column of $A$ is called an _atom_.  We seek _overcomplete_ dictionaries so that $n<p$, and to make the problem more tractable, we should have even more data than we have atoms so that $n<p<m$.  

> **Greedy Algorithms**: To solve 
$$
\text{min}_{A, Z} \left|\left|Y-AZ\right|\right|_{F} + \lambda \left|\left|Z\right|\right|_{1}
$$
we make an initial guess for $A_{0}$ and then solve in two stages:
\begin{align*}
\text{Find} ~ Z_{1} = & \text{arg min}_{Z} \left|\left|Y-A_{0}Z\right|\right|_{F} + \lambda \left|\left|Z\right|\right|_{1} \\
\text{Find} ~ A_{1} = & \text{arg min}_{A} \left|\left|Y-AZ_{1}\right|\right|_{F}.      
\end{align*}
Repeat then until you get convergence.  


> **Computing $D_{A}\left|\left|Y-AZ\right|\right|_{F}^{2}$**: So if we define the function $f(A)=\left|\left|Y-AZ\right|\right|_{F}^{2}$, then we can find its directional derivative at $A$ with respect to the direction $W$, denoted as $<D_{A}f(A), W>=\text{tr}((D_{A}f(A))^{T}W)$, via the formula:
$$
<D_{A}f(A), W> = \lim_{\epsilon\rightarrow 0}\frac{f(A+\epsilon W) - f(A)}{\epsilon} = \left<-2Z(Y-AZ)^{T}, W\right>
$$
Thus, if we want to find $A_{\ast}$ such that $D_{A}f(A)|_{A=A_{\ast}}=0$, then we see that $A_{\ast}$ solves
$$
Z(Y-A_{\ast}Z)^{T} = 0 \implies A_{\ast}ZZ^{T} = YZ^{T}.
$$

> **Moore-Penrose Pseudoinverse**: If for the $p\times m$ real-valued matrix $Z$ we have $\text{rank}(Z)=p$ (remember $p<m$), then we can find the rank-reduced SVD of $Z$ which is $Z = U\Sigma_{p}V_{p}^{T}$ where $\Sigma_{p}$ is $p\times p$ and has strictly positive diagonal entries so $\Sigma_{p}^{-1}$ exists.  Our full rank condition ensures that $U^{T}U=UU^{T}=I$, but we only have $V_{p}^{T}V_{p}=I$ with $V_{p}V_{p}^{T}$ in general being a projection matrix, not the identity. We then see that 
$$
ZZ^{T} = U\Sigma_{p}^{2}U_{p}^{T}
$$
is of full rank and thus invertible.  Therefor the problem
$$
A_{\ast}ZZ^{T} = YZ^{T}
$$
can now be written as 
$$
A_{\ast} = YZ^{-P} = YV_{p}\Sigma_{p}^{-1}U^{T}.
$$

**Problem 1** (Graduate): Since there was so much interest in this problem, I'll have y'all work it out.  So, suppose that, for ${\bf y}, {\bf z} \in \mathbb{R}^{n}$ we want to solve 

$$
{\bf z}_{\ast} = \text{arg min}_{{\bf z}} ~ \left|\left|{\bf y} - A{\bf z}\right|\right|_{2}^{2} + \lambda \left|\left|{\bf z}\right|\right|_{1}
$$

where $\lambda > 0$ and for real-valued, square $A$, $A^{T}A = AA^{T} = I$.  Letting ${\bf u} = A^{T}{\bf y}$, show that 

$$
z_{\ast,l} = \left\{
\begin{array}{rl}
u_{l} - \frac{\lambda}{2} & u_{l} > \frac{\lambda}{2} \\
0 & |u_{l}| \leq \frac{\lambda}{2} \\ 
u_{l} + \frac{\lambda}{2} & u_{l} < -\frac{\lambda}{2} 
\end{array}
\right.
$$

Show that this multi-part formula can be written in the freakishly compact form

$$
z_{\ast,l} = u_{l}\text{max}\left\{0, 1 - \frac{\lambda}{2|u_{l}|}\right\}
$$

Note, to do all of this, you'll need the sub-differential of $|x|$, $\partial |x|$, where 

$$
\partial |x| = \left\{
\begin{array}{rl}
1 & x > 0\\
r\in(-1,1)& x=0, ~ \\
-1 & x < 0
\end{array}
\right.
$$

### Commentary on Problem 1

So again, the big point with this problem is that it gives us a computabible example in which we see how LASSO regularization encourages sparsity.  

**Problem 2**: So guilty confession time, I've been playing a bit fast and loose with norms during lecture, so let's clear that issue up.  First off, for the Frobenius norm, where 

$$
\left|\left|A\right|\right|^{2}_{F} = \text{tr}\left(A^{T}A\right) = \text{tr}\left(A A^{T}\right) = \sum_{j=1}^{N_{r}}\sum_{k=1}^{N_{c}}\left|a_{ij}\right|^{2}
$$

* Show for $n\times m$ $Y$, $n\times p$ $A$, and $p\times m$ $Z$ that  
$$
\left|\left|Y - AZ\right|\right|^{2}_{F} = \sum_{l=1}^{m}\left|\left|{\bf y}_{l} - A{\bf z}_{l}\right|\right|^{2}_{2},
$$
where ${\bf y}_{l}$ and ${\bf z}_{l}$ denote the columns of $Y$ and $Z$ respectively.  

So this result shows that any Frobenius norm problem can be thought of essentially as an average across the respective columns of the matrices $Y$ and $Z$ relative to the matrix $A$.  This is why you'll see me swap back and forth between $F$ and the 2-norm without pausing too much.  

My next sin was in writing $\left|\left|Z\right|\right|_{1}$ for general $p\times m$ matrices $Z$.  We've only defined $\left|\left|\cdot\right|\right|_{1}$ for vectors.  To generalize this to matrices, we define

$$
\left|\left|Z\right|\right|_{1} = \text{sup}_{{\bf x}\neq 0} \frac{\left|\left|Z{\bf x}\right|\right|_{1}}{\left|\left|{\bf x}\right|\right|_{1}}
$$

* For ${\bf x}\in \mathbb{R}^{m}$ show that 
    $$
    \left|\left|Z{\bf x}\right|\right|_{1}\leq \left(\text{max}_{1\leq l \leq m}\left|\left|{\bf z}_{l}\right|\right|_{1}\right) \left|\left|{\bf x}\right|\right|_{1},
    $$
    where again ${\bf z}_{l}$ denotes the $l^{th}$-column of $Z$.  Thus we see that 
    $$
    \left|\left|Z\right|\right|_{1} \leq \text{max}_{1\leq l \leq m}\left|\left|{\bf z}_{l}\right|\right|_{1}
    $$
    
    
* WOLOG, suppose that $\text{max}_{1\leq l \leq m}\left|\left|{\bf z}_{l}\right|\right|_{1} = \left|\left|{\bf z}_{1}\right|\right|_{1}$.  Show that if we choose ${\bf x} = (1 ~0 \cdots 0)^{T}$ that 
$$
\left|\left|Z{\bf x}\right|\right|_{1}  = \text{max}_{1\leq l \leq m}\left|\left|{\bf z}_{l}\right|\right|_{1},
$$
and thus 
$$
\left|\left|Z\right|\right|_{1} = \text{max}_{1\leq l \leq m}\left|\left|{\bf z}_{l}\right|\right|_{1}
$$

### Commentary on Problem 2

Thus, what you are showing in Problem 2 is that if we want to solve LASSO related problems like

$$
\min_{Z} \left|\left|Y - AZ\right|\right|_{F}, ~ \text{with}~\left|\left|Z\right|\right|_{1}\leq \epsilon,
$$

then this is equivalent to optimizing column-by column, i.e. for all $1\leq l \leq m$, find 

$$
\min_{{\bf z}_{l}} \left|\left|{\bf y}_{l} - A{\bf z}_{l}\right|\right|_{2}, ~ \text{with}~ \left|\left|{\bf z}_{l}\right|\right|_{1}\leq \epsilon,
$$

and vice versa. 

**Problem 3**: So it shouldn't be surprising that dictionary learning is something that you can do via library calls in Scikit-Learn.  To wit, there is a full tutorial that walks you through the process of using dictionary learning to do image denoising at:

[Noisy Raccoon](https://scikit-learn.org/stable/auto_examples/decomposition/plot_image_denoising.html#sphx-glr-auto-examples-decomposition-plot-image-denoising-py)

As you can readily see, you can (and should) download the Jupyter notebook that implements the process soup-to-nuts.  So me making you recreate all of it is... silly.  

That said, I can ask you questions that hopefully make you crawl through the code line-by line, and of course there is always theory to fuss over, so let's get to it.  Note, please make all changes to code in that notebook, but please report results in this one.  Upload both to your homework folder.  

Anyway, in the code, dictionary learning is done via the function `MiniBatchDictionaryLearning()` which solves:

$$
\min_{A, Z} \left|\left|Y - AZ\right|\right|_{F} + \lambda \left|\left|Z\right|\right|_{1, 1}
$$

where 

$$
\left|\left|Z\right|\right|_{1, 1} = \sum_{j=1}^{p}\sum_{l=1}^{m}|z_{jl}|
$$

* Show that if $\left|\left|Z\right|\right|_{1, 1} < \epsilon$ then $\left|\left|Z\right|\right|_{1} < \epsilon$.  Likewise, show that if $\left|\left|Z\right|\right|_{1} < \epsilon$ then $\left|\left|Z\right|\right|_{1, 1} < m\epsilon$.  This shows that we aren't introducing anything radically new by using $\left|\left|Z\right|\right|_{1, 1}$.

* In the greedy algorithm approach to solving the dictionary learning problem, given that we find $A_{1}=YZ_{1}^{-P}$, show that 
$$
\left|\left|Y - A_{1}Z_{1}\right|\right|_{F} = \left|\left|Y (I - V_{1,p}V_{1,p}^T) \right|\right|_{F} = \left|\left|(I - V_{1,p}V_{1,p}^T) Y\right|\right|_{F}
$$
How could you interpret this result?  

* In the downloaded notebok, keeping the number of atoms used fixed, change the patch size to $14\times 14$.  Report any differences in the results and try to explain them.  
* In the downloaded notebok, keeping the number of atoms used fixed, change the patch size to $3\times 3$.  Report any differences in the results and try to explain them.  
* In the downloaded notebok, keeping the patch size fixed to 7, change the number of atoms used to 1 and 10 (instead of 2).  Report any differences in the results and try to explain them.  
* In the downloaded notebok, keeping the patch size fixed to 7, and using the original number of atoms, change the batch size to 300.  Report any differences in the results and try to explain them.  
* Relative to the prior four problems, what parameter choices give you the smallest difference in the image denoising?  Explain your reasoning behind your choices.  